In [ ]:
%load_ext blackcellmagic

#### Imports, setup and helper functions

In [ ]:
import numpy as np
import pandas as pd
import deepdish as dd
import h5py
import nibabel.freesurfer.mghformat as mgh
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import ttest_rel

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES, ROI_COLORS, DVA_PER_IMAGE
from spacestream.core.paths import DATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem
from spacestream.utils.get_utils import get_mapping
from spacestream.utils.array_utils import get_flat_indices

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
t_threshold=3
checkpoint = "checkpoint0"

In [ ]:
def retinotopy(dims):
    """Polar angle of each unit, relative to the center of the tissue map"""
    flat_indices = get_flat_indices(dims)
    x_rfs = flat_indices.x_flat
    y_rfs = flat_indices.y_flat

    x_diff = x_rfs - np.mean(x_rfs)
    y_diff = y_rfs - np.mean(y_rfs)

    eccentricity = np.sqrt(x_diff**2 + y_diff**2)
    polar_angle = np.arctan2(y_diff, x_diff)

    # convert eccentricity to dva
    num_taps = dims[-1]
    dva_per_tap = DVA_PER_IMAGE / num_taps
    eccentricity = eccentricity * dva_per_tap

    # convert polar_angle to degrees
    polar_angle = np.degrees(polar_angle)

    return (eccentricity, polar_angle)

#### Load and format data

In [ ]:
# eccentricity by unit
dims = (512, 7, 7)
eccentricity, polar_angle = retinotopy(dims)

In [ ]:
# model unit selectivity
sel = np.load(RESULTS_PATH + "analyses/selectivity/20230811_sel_arrays.npz")
meta = pd.read_csv(RESULTS_PATH + "analyses/selectivity/20230811_sel_meta.csv")

faces = meta[(meta["contrast"].isin(["Faces"])) & (meta["sel_fn"].isin(["t-stat"]))]

In [ ]:
data = {"subject": [], "hemi": [], "seed": [], "stream": [], "eccen": []}

for seedx, model_seed in enumerate(seeds):
    
    #selectivity
    idx = int(faces[(faces["spatial_weight"].isin([0.25])) & (faces["seed"].isin([seedx]))]['Unnamed: 0'])
    face_selective = np.where(sel.f.arrays[idx]>t_threshold)[0] 

    for hidx, hemi in enumerate(hemis):
        for sidx, subj in enumerate(SUBJECTS):
            subj_name = "subj" + subj

            mapping = get_mapping(
                subj_name,"unit2voxel", "0.25", seedx, 0, hemi, checkpoint
            )

            for idx, stream in enumerate(CORE_ROI_NAMES):
                eccen = np.mean(
                    eccentricity[
                        list(
                            set(face_selective)
                            & set(np.where(mapping["winning_roi"] == 5+idx)[0])
                        )
                    ]
                )

                data["subject"].append(subj)
                data["hemi"].append(hemi)
                data["seed"].append(model_seed)
                data["stream"].append(stream)
                data["eccen"].append(eccen)

In [ ]:
df = pd.DataFrame(data)
df_avg = pd.DataFrame(df.groupby(["subject","stream"])["eccen"].mean()).reset_index() #mean across seeds and hemis

#### Plotting

In [ ]:
ROI_COLORS = ['#377E2C', '#1A1AAC', '#8C1A4C']
ROI_COLORS[1:][::-1]

In [ ]:
sns.set_theme(style="ticks")

fig, ax = plt.subplots(figsize=(4,12))


kwargs = {
    "size": 9,
}
a=sns.stripplot(
    x="stream",
    y="eccen",
    hue="stream",
    jitter = 0.1,
    linewidth = 0.75,
    edgecolor="w",
    palette=ROI_COLORS[1:][::-1],
    data=df[(df["hemi"].isin(["rh"])) & (df["stream"].isin(["Ventral", "Lateral"]))],
    order = ["Lateral", "Ventral"],
    dodge=True,
    **kwargs)

b=sns.stripplot(
    x="stream",
    y="eccen",
    hue="stream",
    jitter = 0.2,
    linewidth = 1,
    edgecolor="w",
    marker = "^",
    palette=ROI_COLORS[1:][::-1],
    data=df[(df["hemi"].isin(["lh"])) & (df["stream"].isin(["Ventral", "Lateral"]))],
    order = ["Lateral", "Ventral"],
    dodge=True,
    **kwargs)
c=sns.violinplot(
    x="stream",
    y="eccen",
    hue="stream",
    fill=True,
    linewidth=3,
    #edgecolors="white",
    inner = 'box',
    #jitter=0.2,
    saturation=0.9,
    palette=ROI_COLORS[1:][::-1],
    data=df[(df["stream"].isin(["Ventral", "Lateral"]))],
    order = ["Lateral", "Ventral"],
    dodge=False)
    #**kwargs)

plt.setp(c.collections, alpha=.85,edgecolor='white')



ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend([],[], frameon=False)

save_path = RESULTS_PATH + "/figures/5a.png"
fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
lateral = df_avg[df_avg["stream"].isin(["Lateral"])]['eccen']
ventral = df_avg[df_avg["stream"].isin(["Ventral"])]['eccen']

In [ ]:
h,p = ttest_rel(lateral, ventral)
print(h,p)

In [ ]:
print(np.mean(lateral), np.std(lateral))
print(np.mean(ventral), np.std(ventral))